In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import seaborn as sns

In [3]:
%run ../src/model_test_lib.py

In [4]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [5]:
train = pd.read_pickle(r'..\data\processed\train_v1.pkl')
test = pd.read_pickle(r'..\data\processed\test_v1.pkl')

In [7]:
features = train.columns.tolist()

In [8]:
target_open = 'is_open'
target_click = 'is_click'
pid = 'id'
campaign_id = 'campaign_id'
user_id = 'user_id'

In [9]:
features.remove(target_open)
features.remove(target_click)
features.remove(pid)
features.remove(campaign_id)
features.remove(user_id)

In [10]:
memory_cleaner()

Allocation: 2.23%
Unreference: -2.23%
Collect: 0.00%
Overall: -0.00%


### Predict all is_open = mode(is_open)

In [ ]:
y = [int(train['is_open'].mode()[0])] * len(train)

In [ ]:
metrics.accuracy_score(train['is_open'], y)

In [ ]:
cmf = metrics.confusion_matrix(train['is_open'], y)
sns.heatmap(cmf, annot=True);

The base accuracy is high because the events are imbalanced. 

### Let's check how some basic models work without any hyperparameters

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
model_names = ['LR', 'DTC']
model_list = [
    LogisticRegression(),
    DecisionTreeClassifier()
]

In [ ]:
test_classifiers(model_names, model_list, train, features, target_open)

##### We can observe that models perform no better than mode prediction. This is because most of the algorithms are written to just maximize the accuracy. Also, Precision and F-score are ill-defined, this is because the True Positives and False Positives for is_open = 0
### Let's tune the hyperparameter class_weight to be balanced, so that the algorithm treats them equal

In [11]:
model_names = ['LR', 'DTC', 'MLP']
model_list = [
    LogisticRegression(class_weight='balanced'),
    DecisionTreeClassifier(class_weight='balanced'),
    MLPClassifier(
                        hidden_layer_sizes=(1024, ) * 4,
                        activation='relu',
                        solver='adam',
                        alpha=0.0001,
                        batch_size=100,
                        max_iter=64,
                        learning_rate_init=0.001,
                        random_state=42,
                        early_stopping=True,
                        verbose=True
    )
]

In [ ]:
test_classifiers(model_names, model_list, train, features, target_open)

#### Let's use only few features and not all

In [ ]:
features

In [12]:
original_features = ['no_of_images', 'no_of_internal_links', 'no_of_sections', 'total_links']
derived_feature_external_link = ['no_of_external_links']
time_features = ['day_of_week', 'time_group']
communication_features = [col for col in features if col.startswith('communication_')]
binned_features = [col for col in features if col.startswith('bin_')]
sub_features = [col for col in features if col.startswith('sub -')]
body_features = [col for col in features if col.startswith('body -')]

In [13]:
# Test to check if newly grouped feature contains all the features
set(features) == set(original_features + derived_feature_external_link + time_features + communication_features 
                     + binned_features + sub_features + body_features)

True

#### Let's not include the original feature and the external link features as they are already part of binned features. And create a seprate group of features that has at least 0.05 event distinguishing capability

In [17]:
# Not including body, as user cannot see the body before opening the email
new_feature_group = time_features + communication_features + binned_features + sub_features + body_features

In [20]:
selected_features = []
for feat in new_feature_group:
    group = (train.groupby(feat)['is_open'].sum() / train.groupby(feat)['is_open'].count())
    print(group)
    if len(group) == 2:
        present = group.get_values()[1]
        absent = group.get_values()[0]
        
        if abs(present - absent) >= 0.05:
            selected_features.append(feat)

day_of_week
0   0.099
1   0.058
2   0.146
3   0.154
4   0.077
5   0.057
6   0.072
Name: is_open, dtype: float64
time_group
0   0.063
1   0.109
2   0.099
3   0.102
Name: is_open, dtype: float64
communication_type_Conference
0   0.106
1   0.090
Name: is_open, dtype: float64
communication_type_Corporate
0   0.100
1   0.132
Name: is_open, dtype: float64
communication_type_Hackathon
0   0.094
1   0.290
Name: is_open, dtype: float64
communication_type_Newsletter
0   0.114
1   0.084
Name: is_open, dtype: float64
communication_type_Others
0   0.095
1   0.201
Name: is_open, dtype: float64
communication_type_Upcoming Events
0   0.102
1   0.092
Name: is_open, dtype: float64
communication_type_Webinar
0   0.100
1   0.352
Name: is_open, dtype: float64
bin_no_of_external_links_1-3
0   0.102
1   0.092
Name: is_open, dtype: float64
bin_no_of_external_links_3-6
0   0.071
1   0.119
Name: is_open, dtype: float64
bin_no_of_external_links_6-9
0   0.104
1   0.059
Name: is_open, dtype: float64
bin_no_of_exte

body - dear avian
0   0.091
1   0.105
Name: is_open, dtype: float64
body - dear hacker
0   0.100
Name: is_open, dtype: float64
body - deep
0   0.140
1   0.053
2   0.052
Name: is_open, dtype: float64
body - deep learn
0   0.140
1   0.053
2   0.052
Name: is_open, dtype: float64
body - dr kirk
0   0.105
1   0.056
2   0.127
3   0.132
Name: is_open, dtype: float64
body - eager know
0   0.102
1   0.092
Name: is_open, dtype: float64
body - earli
0   0.102
1   0.046
2   0.133
3   0.122
Name: is_open, dtype: float64
body - earli bird
0   0.102
1   0.046
2   0.133
3   0.122
Name: is_open, dtype: float64
body - end date
0   0.100
Name: is_open, dtype: float64
body - engin
0   0.098
1   0.265
Name: is_open, dtype: float64
body - event
0   0.094
1   0.113
2   0.142
3   0.090
4   0.083
Name: is_open, dtype: float64
body - event competit
0   0.102
1   0.092
Name: is_open, dtype: float64
body - excit
0   0.094
1   0.093
2   0.053
4   0.201
Name: is_open, dtype: float64
body - excit event
0   0.096
1  

body - world
0   0.101
1   0.171
2   0.068
Name: is_open, dtype: float64
body - year
0   0.090
1   0.061
2   0.171
Name: is_open, dtype: float64


In [19]:
selected_features

['communication_type_Hackathon',
 'communication_type_Others',
 'communication_type_Webinar',
 'bin_no_of_external_links_9-12',
 'bin_no_of_images_1-3',
 'bin_no_of_images_12-15',
 'bin_no_of_images_15-18',
 'bin_no_of_images_18-21',
 'bin_no_of_images_3-6',
 'bin_no_of_images_6-9',
 'bin_no_of_images_9-12',
 'bin_no_of_internal_links_1-30',
 'bin_no_of_internal_links_>100',
 'sub - 2018',
 'sub - ai',
 'sub - data scientist',
 'sub - datahack',
 'sub - datahack summit',
 'sub - day',
 'sub - go',
 'sub - hackathon meetup',
 'sub - hackathon win',
 'sub - inr',
 'sub - opportun',
 'sub - scientist',
 'sub - summit',
 'sub - win',
 'sub - win prize',
 'body - 2018',
 'body - analyt onlin',
 'body - build',
 'body - come part',
 'body - contest',
 'body - date',
 'body - engin',
 'body - get readi',
 'body - global',
 'body - hire',
 'body - hire hackathon',
 'body - interview',
 'body - learn data',
 'body - learn deep',
 'body - learn machin',
 'body - learn upcom',
 'body - mckinsey a

In [ ]:
# Retest the same balanced model
test_classifiers(model_names, model_list, train, selected_features, target_open)

In [ ]:
test_classifiers_roc_auc_score(model_names, model_list, train, selected_features, target_open)

In [ ]:
len(new_feature_group)

In [ ]:
selected_features = select_features(train, new_feature_group, target_open, k=35)

In [ ]:
test_classifiers_roc_auc_score(model_names, model_list, train, selected_features, target_open)

In [18]:
# model_names = ['LR', 'DTC']
# model_list = [
#     LogisticRegression(class_weight='balanced', max_iter=1000),
#     DecisionTreeClassifier(class_weight='balanced', min_samples_split=0.05)
# ]
test_classifiers_roc_auc_score(model_names, model_list, train, new_feature_group, target_open)

Allocation: 2.20%
Unreference: -2.20%
Collect: 0.00%
Overall: 0.00%
LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
ROC AUC Score : 0.6989
Cross-Validation ROC AUC Score : 0.3699


Allocation: 2.74%
Unreference: -2.67%
Collect: 0.00%
Overall: 0.06%
DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
ROC AUC Score : 0.699
Cross-Validation ROC AUC Score : 0.4507


Allocation: 2.86%
Unreference: -2.67%
Collect: 0.00%
Overall: 0.19%
MLPClassifier(activation='relu', alpha=0

MemoryError: 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(train[new_feature_group], train[target_open], test_size=0.3, random_state=42)

In [ ]:
model = DecisionTreeClassifier(class_weight='balanced', min_samples_split=0.05)

In [ ]:
model.fit(train_X, train_y)

In [ ]:
pred_proba = model.predict_proba(test_X)

In [ ]:
metrics.roc_auc_score(test_y, pred_proba[:,1])

### LGBM

In [ ]:
import lightgbm as lgb

In [ ]:
# set the data in format lgb accepts
# d_train = lgb.Dataset(train_X, label=train_y)
d_train = lgb.Dataset(train[new_feature_group], label=train[target_open])

In [ ]:
# set parameters
## you can tune the parameters can try to better score

params = {'task': 'train',
    'boosting_type': 'dart',
    'objective': 'binary',
    'is_unbalance': True,
    'metric': 'auc',
    'learning_rate': 0.1, 
    'max_depth': 51, 
    'num_leaves': 175, 
    'feature_fraction': 0.5, 
    'max_bin': 256,
    'bagging_fraction': 0.8,
    'bagging_freq': 3
}

In [ ]:
lgb_cv = lgb.cv(params, d_train, num_boost_round=1000, nfold= 5, shuffle=True, stratified=True,
                verbose_eval=20, early_stopping_rounds=500)

In [ ]:
## get nround value which had lowest error
nround_max = lgb_cv['auc-mean'].index(np.max(lgb_cv['auc-mean']))
nround_min = lgb_cv['auc-mean'].index(np.min(lgb_cv['auc-mean']))
print("MAX Rounds = " + str(nround_max) + " & max auc: " + str(np.max(lgb_cv['auc-mean'])))
print("MIN Rounds = " + str(nround_min) + " & min auc: " + str(np.min(lgb_cv['auc-mean'])))

In [ ]:
## train the model
model = lgb.train(params, d_train, num_boost_round=nround_max)

In [ ]:
import pickle
filename = r'..\models\model_lgbm_open_v1.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# Read and predict
model = pickle.load(open(filename, 'rb'))

In [ ]:
pred = model.predict(train[new_feature_group])

In [ ]:
pred[:5]

In [ ]:
best_ac = -1
best_i = -1
for i in np.arange(0.350, 0.800, 0.001):
    # ac = metrics.accuracy_score(test_y, np.where(preds > i, 1, 0))
    ac = metrics.roc_auc_score(train[target_open], np.where(pred > i, 1, 0))
    print('i = ' + str(i) + ', ac = ' + str(ac))
    if ac > best_ac:
        best_ac = ac
        best_i = i

In [ ]:
print(best_ac)
print(best_i)

In [ ]:
pred = model.predict(test[new_feature_group])

In [ ]:
pred = np.where(pred > 0.5, 1, 0)

In [ ]:
pred.sum() / len(pred)

In [ ]:
test['is_open'] = pred

In [ ]:
test.to_pickle(r'..\data\processed\test_open_pred_v1.pkl')